In [12]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession,SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Row
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window
from pyspark.ml.feature import StopWordsRemover, RegexTokenizer,HashingTF
from pyspark.ml import Pipeline
import re
from nltk.stem import PorterStemmer
from pyspark.ml.linalg import Vectors
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.clustering import LDA
from pyspark.ml.feature import IDF

In [3]:
# Initialise Spark Session
spark = SparkSession.builder.appName("Experiment3").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [41]:
#PREPARING DATAFRAMES FOR DATSETS

#Authors Dataframe
#df_authors = spark.read.csv("Datasets/authors.csv", sep = ",", header = True, quote = '"')

#PaperCsv dataframe
papersCsvSchema = StructType([
    StructField("paper_id",StringType(),False),
    StructField("type",StringType(),False),
    StructField("journal",StringType(),False),
    StructField("book_title",StringType(),False),
    StructField("series",StringType(),False),
    StructField("publisher",StringType(),False),
    StructField("pages",StringType(),False),
    StructField("volume",StringType(),False),
    StructField("number",StringType(),False),
    StructField("year",StringType(),False),
    StructField("month",StringType(),False),
    StructField("postedat",StringType(),False),
    StructField("address",StringType(),False),
    StructField("title",StringType(),False),
    StructField("abstract",StringType(),False),
])
df_paperCsv = spark.read.csv("Datasets/papers.csv", sep = ",", header = False, schema = papersCsvSchema, quote = '"')

#UserLibrary dataframe
userLibrarySchema = StructType([
    StructField("user_hash_id",StringType(),False),
    StructField("user_library",StringType(),False)
])
df_userLibrary = spark.read.csv("Datasets/users_libraries.txt", sep = ";", header = False, schema = userLibrarySchema)
df_userLibrary = df_userLibrary.selectExpr("user_hash_id","split(user_library,',') AS user_library")

"""
#Keywords dataframe

keywordsSchema = StructType([
    StructField("paper_id",StringType(),False),
    StructField("keyword",StringType(),False)
])

df_keywords = spark.read.csv("Datasets/keywords.csv", sep = ",", header = True, schema = keywordsSchema, quote = '"')

#Terms Dataframe
terms_df = spark.read.csv("Datasets/terms.txt", header = True)

#Stopword Broadcast
stopWords = sc.textFile("Datasets/stopwords_en.txt")
stopWordsBroadcast = sc.broadcast(stopWords.collect())

#Papers Terms Dataframe
def parse_papers_count(line):

    if not line:
        return dict()
    papers_count_raw = line.split(' ')
    papers_count = dict()
    for pcRaw in papers_count_raw:
        paper, count = pcRaw.split(':')
        papers_count[paper] = int(count)
    return papers_count

papers_vocab = spark.read.format("csv").option("header", "true").load("Datasets/papers_terms.txt").rdd
papers_vocab = papers_vocab.map(lambda x: (x[0], parse_papers_count(x[1]))).toDF().selectExpr('_1 AS paper_id','_2 AS term_count')    

"""

'\n#Keywords dataframe\n\nkeywordsSchema = StructType([\n    StructField("paper_id",StringType(),False),\n    StructField("keyword",StringType(),False)\n])\n\ndf_keywords = spark.read.csv("Datasets/keywords.csv", sep = ",", header = True, schema = keywordsSchema, quote = \'"\')\n\n#Terms Dataframe\nterms_df = spark.read.csv("Datasets/terms.txt", header = True)\n\n#Stopword Broadcast\nstopWords = sc.textFile("Datasets/stopwords_en.txt")\nstopWordsBroadcast = sc.broadcast(stopWords.collect())\n\n#Papers Terms Dataframe\ndef parse_papers_count(line):\n\n    if not line:\n        return dict()\n    papers_count_raw = line.split(\' \')\n    papers_count = dict()\n    for pcRaw in papers_count_raw:\n        paper, count = pcRaw.split(\':\')\n        papers_count[paper] = int(count)\n    return papers_count\n\npapers_vocab = spark.read.format("csv").option("header", "true").load("Datasets/papers_terms.txt").rdd\npapers_vocab = papers_vocab.map(lambda x: (x[0], parse_papers_count(x[1]))).toDF(

In [7]:
#Cleaning and Tokenizing the data
def phraseTokenization(x):
    rawPhrase = x[13] + " " + x[14] #concatenating title and abstract
    rawPhrase = rawPhrase.replace("-","") #removing - from phrase
    rawPhrase = rawPhrase.replace("_","") #removing _ from phrase
    rawPhrase = rawPhrase.strip() #removing any trailing or leading whitespaces
    
    #spliting phrase based on non-alphaNumeric characters
    phraseArray = re.split('[^a-zA-Z0-9]+',rawPhrase) 
    
    #remove words with less than 3 char
    phraseArrayFilteredWords = [i for i in phraseArray if len(i) >= 3]
    
    return (x[0],list(phraseArrayFilteredWords))


df_tokenize = df_paperCsv.na.fill(value="").rdd.map(phraseTokenization).toDF()

#Removing StopWords using ML
swRemover = StopWordsRemover(inputCol="_2", outputCol="cleaned_terms")
df_cleanedData = swRemover.transform(df_tokenize)
df_cleanedData = df_cleanedData.selectExpr("_1 AS paper_id","cleaned_terms")

#Stemming using Porter stemmer Algo
ps =  PorterStemmer()

def stemmingTerms(x):
    stemmedWords = []
    for word in x:
        rootWord = ps.stem(word)
        stemmedWords.append(rootWord)
    return stemmedWords

df_cleanedData = df_cleanedData.rdd.mapValues(stemmingTerms).toDF().selectExpr("_1 AS paper_id","_2 AS cleaned_terms")

In [8]:
# Find the count of papers in which the term is present
df_paperCount = df_cleanedData.selectExpr("paper_id","explode(cleaned_terms) AS terms").distinct().groupBy("terms").count().withColumnRenamed("count", "paper_count")

#10 percent of total papers present in file
noOfDistinctPapers_df = int(df_cleanedData.select(countDistinct("paper_id")).collect()[0][0])
tenPercentOfTotalPapers = int(noOfDistinctPapers_df/10)

# remove words appear in more than 10% of the papers and keep only the words that appear in at least 20 papers 
df_filterdTerms = df_paperCount.filter((df_paperCount["paper_count"]<=tenPercentOfTotalPapers) & (df_paperCount["paper_count"]>=20))

#Fetch top 1000 terms 
top1000Terms = df_filterdTerms.orderBy(col("paper_count").desc()).limit(1000)

In [9]:
#associate unique integer values to each term
df_termsWithUniqueIndex = top1000Terms.withColumn("unique_index",row_number().over(Window.orderBy("paper_count"))).selectExpr("terms","unique_index-1 AS unique_index")

#Collect all terms in a list
terms_collection =  [int(row.terms) for row in df_termsWithUniqueIndex.collect()]

# Generating Termfrequency Vector for each paper
df_cleanedDataExplode = df_cleanedData.selectExpr("paper_id","explode(cleaned_terms) AS terms")

#Getting the Unique_index of term 
df_cleanedDataJoinIndex = df_cleanedDataExplode.join(df_termsWithUniqueIndex,df_cleanedDataExplode.terms == df_termsWithUniqueIndex.terms , how = "inner").select(df_cleanedDataExplode.paper_id,df_cleanedDataExplode.terms,df_termsWithUniqueIndex.unique_index)

#Getting the term_frequency in each paper
df_cleanedDataJoinIndex = df_cleanedDataJoinIndex.groupBy("paper_id","unique_index").count().withColumnRenamed("count", "term_frquency")

#Creating a sparseVector respresentation for each paper
rdd_CleanedDataReducedByPaperId = df_cleanedDataJoinIndex.rdd.map(lambda x: (x[0], [(x[1], x[2])])).reduceByKey(lambda a, b: a + b)
rdd_CleanedDataReducedByPaperId = rdd_CleanedDataReducedByPaperId.map(lambda x: (x[0],Vectors.sparse(1000,x[1])))

df_CleanedDataSparseVector = rdd_CleanedDataReducedByPaperId.toDF().selectExpr("_1 AS paper_id","_2 AS term_frequency_vector")

In [10]:
df_CleanedDataSparseVector.take(1)

[Row(paper_id='498902', term_frequency_vector=SparseVector(1000, {33: 3.0, 47: 1.0, 79: 1.0, 97: 1.0, 138: 1.0, 170: 6.0, 354: 1.0, 368: 1.0, 394: 1.0, 482: 2.0, 491: 1.0, 541: 1.0, 550: 1.0, 566: 1.0, 581: 1.0, 596: 1.0, 622: 1.0, 632: 1.0, 663: 1.0, 670: 1.0, 720: 2.0, 723: 1.0, 762: 1.0, 764: 1.0, 773: 1.0, 797: 1.0, 820: 1.0, 826: 1.0, 837: 2.0, 843: 1.0, 879: 1.0, 881: 1.0, 890: 1.0, 892: 1.0, 894: 1.0, 928: 1.0, 937: 1.0, 946: 2.0, 949: 3.0, 952: 1.0, 962: 1.0, 965: 1.0, 984: 1.0, 985: 4.0, 990: 1.0, 992: 1.0}))]

In [13]:
#TF-IDF Representation for each paper

idf = IDF(inputCol="term_frequency_vector", outputCol="tf_idf_vector")
tf_idf_model = idf.fit(df_CleanedDataSparseVector)
df_rescaledCleanedData = tf_idf_model.transform(df_CleanedDataSparseVector)
df_rescaledCleanedData = df_rescaledCleanedData.select("paper_id", "tf_idf_vector")

In [14]:
df_rescaledCleanedData.take(1)

[Row(paper_id='498902', tf_idf_vector=SparseVector(1000, {33: 13.8439, 47: 4.596, 79: 4.5494, 97: 4.5325, 138: 4.4866, 170: 26.6474, 354: 4.1522, 368: 4.1302, 394: 4.0846, 482: 7.8051, 491: 3.8889, 541: 3.7983, 550: 3.7834, 566: 3.7571, 581: 3.7152, 596: 3.6986, 622: 3.6484, 632: 3.6256, 663: 3.5658, 670: 3.5395, 720: 6.7871, 723: 3.3875, 762: 3.2929, 764: 3.2917, 773: 3.2717, 797: 3.2131, 820: 3.1488, 826: 3.1306, 837: 6.153, 843: 3.0651, 879: 2.9588, 881: 2.9516, 890: 2.9076, 892: 2.9043, 894: 2.8955, 928: 2.7393, 937: 2.6945, 946: 5.2459, 949: 7.8223, 952: 2.5913, 962: 2.5231, 965: 2.5148, 984: 2.3717, 985: 9.4584, 990: 2.328, 992: 2.3177}))]

In [16]:
# Latent Direchlet Allocation

num_topics = 40

# Transform data into LDA supported format
rdd_lda_format = df_CleanedDataSparseVector.rdd.mapValues(Vectors.fromML).map(lambda x: [int(x[0]),x[1]])

#Train the LDA Model
lda_model = LDA.train(rdd_lda_format, k=num_topics)

In [17]:
#Find the top 5 term of each extracted Topic
topicIndices = sc.parallelize(lda_model.describeTopics(maxTermsPerTopic = 5))

def Get_TopicTerms(topic):
    termsId_array = topic[0] 
    terms_array = []
    
    #Finding the term from the their corresponding Unique index
    for i in range(5):
        term = terms_collection[termsId_array[i]]
        terms_array.append(term)
    return terms_array

topics_Final = topicIndices.map(Get_TopicTerms).collect()


In [40]:
topics_final

[['gene', 'protein', 'network', 'interact', 'activ'],
 ['network', 'gene', 'protein', 'interact', 'activ'],
 ['gene', 'protein', 'network', 'sequenc', 'cell'],
 ['gene', 'protein', 'network', 'sequenc', 'interact'],
 ['gene', 'protein', 'network', 'cell', 'sequenc'],
 ['gene', 'protein', 'network', 'cell', 'sequenc'],
 ['gene', 'protein', 'network', 'sequenc', 'cell'],
 ['gene', 'network', 'protein', 'sequenc', 'cell'],
 ['gene', 'protein', 'network', 'cell', 'activ'],
 ['gene', 'protein', 'network', 'cell', 'activ'],
 ['gene', 'network', 'protein', 'sequenc', 'interact'],
 ['gene', 'network', 'protein', 'activ', 'sequenc'],
 ['gene', 'network', 'protein', 'interact', 'activ'],
 ['gene', 'protein', 'network', 'cell', 'activ'],
 ['network', 'gene', 'protein', 'interact', 'activ'],
 ['gene', 'network', 'protein', 'activ', 'interact'],
 ['gene', 'network', 'protein', 'cell', 'activ'],
 ['gene', 'network', 'protein', 'cell', 'activ'],
 ['network', 'gene', 'protein', 'activ', 'interact'],
 

In [50]:
#User Profiling

df_userLibrary_explode = df_userLibrary.selectExpr("user_hash_id","explode(user_library) AS paper_id")
df_userJoined_TfIdf = df_userLibrary_explode.join(df_rescaledCleanedData,df_userLibrary_explode.paper_id == df_rescaledCleanedData.paper_id, how="inner").select(df_userLibrary_explode.user_hash_id,df_userLibrary_explode.paper_id,df_rescaledCleanedData.tf_idf_vector)

In [51]:
df_userJoined_TfIdf.show()

+--------------------+--------+--------------------+
|        user_hash_id|paper_id|       tf_idf_vector|
+--------------------+--------+--------------------+
|21cc6e1f99fae502d...|  498902|(1000,[33,47,79,9...|
|cfaa315e63b14e6d3...|  498902|(1000,[33,47,79,9...|
|9ccd57a8d991f3d7c...|  498902|(1000,[33,47,79,9...|
|c8612ac693308780b...|  498902|(1000,[33,47,79,9...|
|a73ea3f829fde4c26...|  498902|(1000,[33,47,79,9...|
|93833fb937c3b5ecc...|  498902|(1000,[33,47,79,9...|
|bbae15df551ed4a89...|  498902|(1000,[33,47,79,9...|
|40ee6b4e4fefb7bd3...|  498902|(1000,[33,47,79,9...|
|e8e0c7b9dd5799e58...|  498902|(1000,[33,47,79,9...|
|6abfda34cd59fe0d3...|  498902|(1000,[33,47,79,9...|
|b564d6be2c8e64259...|  498902|(1000,[33,47,79,9...|
|6b6c8775389742588...|  498902|(1000,[33,47,79,9...|
|c74188f8efedf5e7c...|  498902|(1000,[33,47,79,9...|
|8f8b77f3ced188fcd...|  498902|(1000,[33,47,79,9...|
|4bdfa5ef59ac63133...|  498902|(1000,[33,47,79,9...|
|511b1eea8a8223a87...|  498902|(1000,[33,47,79